In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

#Tf-idf vectorizer from standard library

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
tfv = TfidfVectorizer(min_df=3, max_df=0.9, max_features=None, strip_accents='unicode',\
               analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,3), use_idf=1,\
               smooth_idf=1, sublinear_tf=1, stop_words='english')

In [ ]:
train, test = pd.read_csv("../input/train.csv"), pd.read_csv("../input/test.csv")
IDs = test['id']
X_Train, X_Test = train['comment_text'], test['comment_text']
#X_test.loc[X_test.isnull()] = " " # replace the 1 NaN value in test
Y_train = train[train.columns[2:]]


In [ ]:
type(X_train)

In [ ]:
import gc

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

import keras
from keras.models import Sequential
from keras.layers import Dense



del train
del test
gc.collect()

print("%.2f of data is not flagged" % (Y_train.loc[(Y_train.sum(axis=1) == 0)].shape[0] / Y_train.shape[0]))


               

print("tfidf-vectorizing train ...")
tfv.fit(X_train)
X_train = tfv.transform(X_train)
print("tfidf-vectorizing test ...")

X_test = tfv.transform(X_test)

print("fitting log reg & reporting cv accuracy ...")
for i in range(Y_train.shape[1]):
    feature = Y_train.columns[i]
    print("\n%s:" % feature)
    print("Baseline: %.2f" % (Y_train.iloc[:,i].sum() / Y_train.shape[0]))
    clf = LogisticRegression(C=4.0, solver='sag')
    clf.fit(X_train, Y_train.iloc[:,i])
    
    print(cross_val_score(clf, X_train, Y_train.iloc[:,i], cv=5, scoring='f1'))
    exec("pred_%s = pd.Series(clf.predict_proba(X_test).flatten()[1::2])" % feature)

submission = pd.DataFrame({
             'id': IDs,
             'toxic': pred_toxic,
             'severe_toxic': pred_severe_toxic,
             'obscene': pred_obscene,
             'threat': pred_threat,
             'insult': pred_insult,
             'identity_hate': pred_identity_hate
             })

#submission.to_csv("submission_logreg.csv", index=False)

****Hashing Vectorizer using a hashing space of 2**18

In [ ]:
tfv=HashingVectorizer(decode_error='ignore', n_features=2 ** 18,
                               alternate_sign=False)

print("tfidf-vectorizing train ...")
tfv.fit(X_train)
X_train = tfv.transform(X_train)
print("tfidf-vectorizing test ...")

X_test = tfv.transform(X_test)

****Prepare Input for Word2vec****

In [ ]:
import gensim

# Load Google's pre-trained Word2Vec model


In [ ]:
train, test = pd.read_csv("../input/train.csv"), pd.read_csv("../input/test.csv")
IDs = test['id']

In [ ]:
X_train=pd.DataFrame(train['comment_text'])
X_test_comments=pd.DataFrame(test['comment_text'])
X_total=pd.concat([X_train,X_test_comments])
X_total_list=list(X_total['comment_text'])
X_train_list=list(X_train['comment_text'])


In [ ]:
import gensim
from gensim.models import Word2Vec

# Load pretrained model (since intermediate data is not included, the model cannot be refined with additional data)
#model2 = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

context_size = 7 # context window length
downsampling = 1e-5 # downsampling for very frequent words
X_total_list_new=[]
for s in X_total_list:
    if type(s) is float:
        s=str(s)
        X_total_list_new.append(s)
    X_total_list_new.append(s)

In [ ]:
model = gensim.models.Word2Vec([s.split() for s in X_total_list_new],size=200, min_count=6,workers=4)

In [ ]:
model_2 = gensim.models.Word2Vec([s.split() for s in X_total_list_new],size=200, min_count=6,workers=4,window = context_size,
    sample = downsampling
)

In [ ]:
model_2

**Most Frequent Words in training data **

In [ ]:
print(model_2.wv.index2word[0], model_2.wv.index2word[1], model_2.wv.index2word[2])

In [ ]:
vocab_size = len(model.wv.vocab)

**Least Frequent word int he vocabulary**

In [ ]:
print(model.wv.index2word[vocab_size-1], model.wv.index2word[vocab_size-2], model.wv.index2word[vocab_size-3])

**Let's Prepare the Feature matrix using the trained features from word2vec on the dictionary**

**First Prepare the embedding matrix and then the feature  matrix**

In [ ]:
len(model.wv.vocab)

In [ ]:
# convert the wv word vectors into a numpy matrix that is suitable for insertion
# into our TensorFlow and Keras models
embedding_matrix = np.zeros((len(model.wv.vocab), 200))
for i in range(len(model.wv.vocab)):
    embedding_vector = model.wv[model.wv.index2word[i]]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
feature_matrix = np.zeros((95851, 200))
i=0
for s in X_train_list:
    #embedding_vector=np.zeros(200)
    initial_vector=np.zeros(200)
    for word in s.split():
    #for i in range(len(model.wv.vocab)):
        if word in model.wv:
            
            embedding_vector=initial_vector+embedding_matrix[model.wv.vocab[word].index]
            initial_vector=embedding_vector
        #embedding
    #if embedding_vector is not None:
    feature_matrix[i] = embedding_vector
    #print("one done")
    i=i+1

In [ ]:
feature_frame=pd.DataFrame(feature_matrix)

In [ ]:
feature_frame.head()

**Prepare the test data set **

In [ ]:
test.head()

In [ ]:
feature_frame=pd.DataFrame(feature_matrix)
X_test_list=list(test['comment_text'])

In [ ]:
test.shape

In [ ]:
# convert the wv word vectors into a numpy matrix that is suitable for insertion
# into our TensorFlow and Keras models

test_feature_matrix = np.zeros((226998, 200))
i=0
for s in X_test_list:
    #embedding_vector=np.zeros(200)
    initial_vector=np.zeros(200)
    if type(s) is float:
        s=str(s)
    for word in s.split():
    #for i in range(len(model.wv.vocab)):
        if word in model.wv:
            
            embedding_vector=initial_vector+embedding_matrix[model.wv.vocab[word].index]
            initial_vector=embedding_vector
        #embedding
    #if embedding_vector is not None:
    test_feature_matrix[i] = embedding_vector
    #print("one done")
    i=i+1

In [ ]:
test_features=pd.DataFrame(test_feature_matrix)

In [ ]:
test_features.shape

**Now lets start with training linear models **

In [ ]:
Y_train

In [ ]:
import gc

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
for i in range(Y_train.shape[1]):
    feature = Y_train.columns[i]
    print("\n%s:" % feature)
    print("Baseline: %.2f" % (Y_train.iloc[:,i].sum() / Y_train.shape[0]))
    clf = LogisticRegression(C=4.0, solver='sag')
    clf.fit(feature_frame, Y_train.iloc[:,i])
    
    print(cross_val_score(clf,feature_frame,Y_train.iloc[:,i], cv=5, scoring='f1'))
    exec("pred_%s = pd.Series(clf.predict_proba(test_features).flatten()[1::2])" % feature)

submission = pd.DataFrame({
             'id': IDs,
             'toxic': pred_toxic,
             'severe_toxic': pred_severe_toxic,
             'obscene': pred_obscene,
             'threat': pred_threat,
             'insult': pred_insult,
             'identity_hate': pred_identity_hate
             })

submission.to_csv("submission_logreg_adarsh.csv", index=False)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm

clf = svm.SVC()
#clf.fit(X, Y)

In [ ]:
print("adastrh")

In [ ]:
Y_train = train[train.columns[2:]]


In [ ]:
print("hello")

In [ ]:
# lets prepare for the SVM classifiers and others
from sklearn.model_selection import cross_val_score
for i in range(Y_train.shape[1]):
    feature = Y_train.columns[i]
    print("\n%s:" % feature)
    print("Baseline: %.2f" % (Y_train.iloc[:,i].sum() / Y_train.shape[0]))
    #clf = LogisticRegression(C=4.0, solver='sag')
    clf.fit(feature_frame, Y_train.iloc[:,i])
    
    print(cross_val_score(clf,feature_frame,Y_train.iloc[:,i], cv=5, scoring='f1'))
    exec("pred_%s = pd.Series(clf.predict_proba(test_features).flatten()[1::2])" % feature)

submission = pd.DataFrame({
             'id': IDs,
             'toxic': pred_toxic,
             'severe_toxic': pred_severe_toxic,
             'obscene': pred_obscene,
             'threat': pred_threat,
             'insult': pred_insult,
             'identity_hate': pred_identity_hate
             })

submission.to_csv("submission_svm_adarsh.csv", index=False)

#Adding Non-linearity 

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=5)


for i in range(0,Y_train.shape[1]):
    for train, test in kfold.split(feature_frame, Y_train.iloc[:,i]):
        print(i)
        classifier = Sequential()
        classifier.add(Dense(output_dim =180, init = 'uniform', activation = 'relu', input_dim = 200))
        classifier.add(Dense(output_dim =180, init = 'uniform', activation = 'relu', input_dim = 200))
        classifier.add(Dense(output_dim =180, init = 'uniform', activation = 'relu', input_dim = 200))
        classifier.add(Dense(output_dim =180, init = 'uniform', activation = 'relu', input_dim = 200))
        classifier.add(Dense(output_dim =180, init = 'uniform', activation = 'relu', input_dim = 200))
        classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))    
        
        classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
        feature = Y_train.columns[i]
        print("\n%s:" % feature)
        print("Baseline: %.2f" % (Y_train.iloc[:,i].sum() / Y_train.shape[0]))
        classifier.fit(feature_frame.iloc[train], Y_train.iloc[:,i].iloc[train], batch_size = 10, nb_epoch = 2)
        scores = classifier.evaluate(feature_frame.iloc[test], Y_train.iloc[:,i].iloc[test], verbose=0)
        print("%s: %.2f%%" % (classifier.metrics_names[1], scores[1]*100))
